Урок 3. Логистическая регрессия. Log Loss

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np

In [191]:
# Исходная функция
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

0 может попасть в логарифм если y_pred будет равен 1 или 0, пытался найти какую нибудь функцию которая при стремлении аргумента к 1 будет бесконечно стремиться к 0 - так и не смог ничего вспомнить ) в итоге предлагаю просто заклампить входные значения с какой нибудь точностью - например в диапазон от 1e-100 до 1-1e-100... реально в пайтоне при современной точности будет иметь значение кламп от 1e-323 до 1-1e-19... дальше уже все округляется в 1 или 0 соответственно.

In [193]:
def calc_logloss(y, y_pred):
    y_pred_clamped = np.clip(y_pred, np.float128(1e-323), np.float128(1)-1e-19)
    err = - np.mean(y * np.log(y_pred_clamped) + (1.0 - y) * np.log(1.0 - y_pred_clamped))
    err = np.sum(err)
    return err

In [141]:
calc_logloss(1, 1)

1.0842021724855044341e-19

In [142]:
calc_logloss(0, 1)

43.668272375276554493

In [143]:
calc_logloss(1, 0)

743.746924740821317

In [144]:
calc_logloss(0, 0)

0.0

Такое - неизящное решение (

1. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [145]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [148]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [149]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [150]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [152]:
X.shape[0]

4

In [156]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [181]:
W = eval_model(X_st, y, iterations=10000, alpha=3.2)

1000 [-24.66166323  -2.58625863  -6.48836871  20.51862732] 0.14515608548614972
2000 [-30.94361661  -3.1748203   -8.28595043  25.51449064] 0.12175660170312452
3000 [-35.62919029  -3.59578876  -9.60752257  29.29589069] 0.10976714676006635
4000 [-39.46740433  -3.93563493 -10.69080562  32.37822401] 0.10176998210815198
5000 [-42.77500088  -4.22300941 -11.62764808  35.02045176] 0.09585970945019102
6000 [-45.72133652  -4.4734476  -12.46641145  37.36113568] 0.09119036317069082
7000 [-48.40697954  -4.69628808 -13.23547143  39.4826709 ] 0.0873266825428291
8000 [-50.8962043   -4.89759487 -13.95275545  41.43786763] 0.08402054986670685
9000 [-53.23234596  -5.0815578  -14.63019366  43.26251651] 0.081119518691459
10000 [-55.44580972  -5.25122724 -15.27602267  44.9819372 ] 0.07852445438043257


Ниже не смог получить ) но даже этот результат думаю не особо полезен - модель мне кажется переобучена в хлам..

2. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [213]:
def calc_pred_proba(X, W):
    result = np.zeros(X.shape[1])
    for n in range(X.shape[1]):
        result[n] = sigmoid(np.dot(X.T[n], W))
    return result

In [214]:
calc_pred_proba(X_st, W)

array([3.20632749e-01, 8.65276587e-04, 1.00000000e+00, 1.52546073e-11,
       9.78121910e-01, 7.02846565e-03, 1.00000000e+00, 4.94947027e-03,
       6.95151970e-01, 1.00000000e+00])

In [221]:
# Если разложить эти вероятности с порогом 0.5 то получим
threshold = lambda x : 1 if x > 0.5 else 0
threshold_vec = np.vectorize(threshold)
threshold_vec(calc_pred_proba(X_st, W))

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [222]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

)) Переобученная модель на тренировочном сете показывает стопроцентное угадывание.

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

Эмм.. Видимо я уже это сделал в предыдущем пункте (у нас ведь всего два класса 0 и 1)

In [237]:
# Запакуем в функцию
def calc_pred(X, W):
    threshold = lambda x : 1 if x > 0.5 else 0
    threshold_vec = np.vectorize(threshold)
    return threshold_vec(calc_pred_proba(X, W))

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

Возможно надо было взять другие данные? На обучающем сете считать все это не имеет смысла же - 100% попадание.
Будет как-то так:

In [249]:
accuracy = np.sum([1 if a == b else 0 for a,b in zip(calc_pred(X_st, W), y)])/y.shape[0]
accuracy

1.0

Что и требовалось...

In [243]:
# Матрица ошибок
m_err = np.zeros((2, 2))
for a, b in zip(calc_pred(X_st, W), y):
    if a == b and a > 0:
        m_err[0][0] += 1
    if a == b and a < 1:
        m_err[1][1] += 1
    if a != b and a > 0:
        m_err[0][1] += 1
    if a != b and a < 1:
        m_err[1][0] += 1

In [244]:
m_err

array([[5., 0.],
       [0., 5.]])

In [245]:
# Точность и полнота
precission = m_err[0][0]/(m_err[0][0] + m_err[1][0])
precission

1.0

In [247]:
recall = m_err[0][0]/(m_err[0][0] + m_err[0][1])
recall

1.0

Скукотища )